In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import preprocess_dataset as ut
from scipy.interpolate import BSpline
from tfkan import DenseKAN
from tensorflow import keras
from keras_tuner import HyperModel, GridSearch, RandomSearch
from tensorflow import keras
from kerastuner.tuners import BayesianOptimization
from sklearn.model_selection import KFold

In [2]:
x_train = tf.convert_to_tensor(pd.read_csv("datasets/x_train.csv").to_numpy()[:, 1:])
y_train = tf.convert_to_tensor(pd.read_csv("datasets/y_train.csv").to_numpy()[:, 1:])

x_val = tf.convert_to_tensor(pd.read_csv("datasets/x_val.csv").to_numpy()[:, 1:])
y_val = tf.convert_to_tensor(pd.read_csv("datasets/y_val.csv").to_numpy()[:, 1:])

x_test = tf.convert_to_tensor(pd.read_csv("datasets/x_test.csv").to_numpy()[:, 1:])
y_test = tf.convert_to_tensor(pd.read_csv("datasets/y_test.csv").to_numpy()[:, 1:])

print(x_train.shape, x_val.shape, x_test.shape, y_train.shape, y_test.shape, y_val.shape)

(60000, 26) (15000, 26) (20000, 26) (60000, 1) (20000, 1) (15000, 1)


In [3]:
#kan_layers = [4,1]
#kan_model = ut.build_model(kan_layers, x_train.shape[1], mlp=False)
#kan_model.summary()
#
#mlp_layers = [20,3,1]
#mlp_model = ut.build_model(mlp_layers, x_train.shape[1], mlp=True)
#mlp_model.summary()
#kan_history = kan_model.fit(x_train, y_train, epochs=20, batch_size=128, validation_data=(x_val, y_val), verbose=1)


In [4]:
## MLP Pura
#
#pure_mlp_layers = [64, 32, 16, 1]
#mlp_model = ut.build_model(pure_mlp_layers, x_train.shape[1], mlp=True, optimizer='rmsprop', loss="categorical_crossentropy")
#mlp_model.summary()
#
#mlp_history = mlp_model.fit(x_train, y_train, epochs=50, batch_size=128, validation_data=(x_test, y_test), verbose=1)

In [8]:
class MyHyperModel(HyperModel):
    def __init__(self, input_shape):
        self.input_shape = input_shape

    def build(self, hp):
        model = keras.Sequential()
        model.add(tf.keras.layers.Input(shape=self.input_shape))
        
        for i in range(hp.Int('num_layers', 2, 5)):
            model.add(DenseKAN(
                units=hp.Int('units_' + str(i), min_value=1, max_value=32, step=1),
                grid_size=hp.Int('grid_size_' + str(i), 3, 7),
                spline_order=hp.Int('spline_order_' + str(i), 2, 4)
            ))
        
        model.add(DenseKAN(1))

        learning_rate = hp.Float('learning_rate', 1e-4, 1e-2, sampling='log')
        optimizer = hp.Choice('optimizer', ['adam', 'adagrad', 'rmsprop', 'adadelta'])
        
        if optimizer == 'adam':
            opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
        elif optimizer == 'adagrad':
            opt = tf.keras.optimizers.Adagrad(learning_rate=learning_rate)
        elif optimizer == 'rmsprop':
            opt = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
        else:
            opt = tf.keras.optimizers.Adadelta(learning_rate=learning_rate)

        model.compile(optimizer=opt,
                      loss='mean_absolute_error',
                      metrics=[
                          tf.keras.metrics.MeanSquaredError(name='mse'),
                          tf.keras.metrics.MeanAbsoluteError(name='mae'),
                          tf.keras.metrics.RootMeanSquaredError(name='rmse')
                      ])
        return model

# Assumo che x_train, y_train siano già definiti
hypermodel = MyHyperModel(input_shape=(26,))

tuner = BayesianOptimization(
    hypermodel,
    objective='val_loss',
    max_trials=50,
    directory=r".\modelli_salvati",
    project_name='retina_kan'
)

tuner.search_space_summary()

# Callbacks
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-6
)

# K-Fold Cross Validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_indices, val_indices) in enumerate(kfold.split(x_train)):
    print(f"Fold {fold + 1}")
    train_indices = tf.convert_to_tensor(train_indices, dtype=tf.int32)
    val_indices = tf.convert_to_tensor(val_indices, dtype=tf.int32)
    
    x_train_fold, x_val_fold = tf.gather(x_train, train_indices), tf.gather(x_train, val_indices)
    y_train_fold, y_val_fold = tf.gather(y_train, train_indices), tf.gather(y_train, val_indices)

    tuner.search(x_train_fold, y_train_fold,
                 epochs=100,
                 validation_data=(x_val_fold, y_val_fold),
                 callbacks=[early_stopping, lr_scheduler])

tuner.results_summary()
best_model = tuner.get_best_models(num_models=1)[0]

# Valutazione del modello migliore sul set di test
# Assumo che x_test, y_test siano già definiti
test_loss, test_mse, test_mae, test_rmse = best_model.evaluate(x_test, y_test)
print(f"Test Loss: {test_loss}")
print(f"Test MSE: {test_mse}")
print(f"Test MAE: {test_mae}")
print(f"Test RMSE: {test_rmse}")

# Salvataggio del modello migliore
best_model.save('best_retina_kan_model.h5')

Trial 50 Complete [00h 11m 02s]
val_loss: 2.200371026992798

Best val_loss So Far: 0.018934112042188644
Total elapsed time: 06h 43m 48s
Fold 2
Fold 3
Fold 4
Fold 5
Results summary
Results in .\modelli_salvati\retina_kan
Showing 10 best trials
Objective(name="val_loss", direction="min")

Trial 17 summary
Hyperparameters:
num_layers: 4
units_0: 8
grid_size_0: 3
spline_order_0: 3
units_1: 21
grid_size_1: 4
spline_order_1: 4
learning_rate: 0.0050089061837515795
optimizer: adam
units_2: 32
grid_size_2: 7
spline_order_2: 2
units_3: 8
grid_size_3: 6
spline_order_3: 2
units_4: 11
grid_size_4: 5
spline_order_4: 2
Score: 0.018934112042188644

Trial 10 summary
Hyperparameters:
num_layers: 5
units_0: 22
grid_size_0: 5
spline_order_0: 2
units_1: 5
grid_size_1: 4
spline_order_1: 3
learning_rate: 0.0003082442963419899
optimizer: adam
units_2: 24
grid_size_2: 5
spline_order_2: 3
units_3: 27
grid_size_3: 7
spline_order_3: 4
units_4: 13
grid_size_4: 7
spline_order_4: 2
Score: 0.023273497819900513

Trial

c:\Users\gobba\miniconda3\envs\kambinete\lib\site-packages\keras\src\saving\saving_lib.py:690: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 32 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0211 - mae: 0.0211 - mse: 0.0371 - rmse: 0.1905


Test Loss: 0.02077632211148739
Test MSE: 0.033895332366228104
Test MAE: 0.020776323974132538
Test RMSE: 0.18410685658454895


In [14]:
# Ottieni i migliori iperparametri
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]

# Ottieni e stampa i nomi e i valori dei parametri
for param_name in best_hyperparameters.values.keys():
    print(f"{param_name}: {best_hyperparameters.get(param_name)}")


num_layers: 4
units_0: 8
grid_size_0: 3
spline_order_0: 3
units_1: 21
grid_size_1: 4
spline_order_1: 4
learning_rate: 0.0050089061837515795
optimizer: adam
units_2: 32
grid_size_2: 7
spline_order_2: 2
units_3: 8
grid_size_3: 6
spline_order_3: 2
units_4: 11
grid_size_4: 5
spline_order_4: 2


In [15]:
# Ottieni i migliori iperparametri per i primi 10 modelli
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=10)

# Per ogni set di iperparametri
for i, hyperparameters in enumerate(best_hyperparameters):
    print(f"Modello {i+1}")
    # Ottieni e stampa i nomi e i valori dei parametri
    for param_name in hyperparameters.values.keys():
        print(f"{param_name}: {hyperparameters.get(param_name)}")
    print("\n")


Modello 1
num_layers: 4
units_0: 8
grid_size_0: 3
spline_order_0: 3
units_1: 21
grid_size_1: 4
spline_order_1: 4
learning_rate: 0.0050089061837515795
optimizer: adam
units_2: 32
grid_size_2: 7
spline_order_2: 2
units_3: 8
grid_size_3: 6
spline_order_3: 2
units_4: 11
grid_size_4: 5
spline_order_4: 2


Modello 2
num_layers: 5
units_0: 22
grid_size_0: 5
spline_order_0: 2
units_1: 5
grid_size_1: 4
spline_order_1: 3
learning_rate: 0.0003082442963419899
optimizer: adam
units_2: 24
grid_size_2: 5
spline_order_2: 3
units_3: 27
grid_size_3: 7
spline_order_3: 4
units_4: 13
grid_size_4: 7
spline_order_4: 2


Modello 3
num_layers: 4
units_0: 4
grid_size_0: 3
spline_order_0: 4
units_1: 23
grid_size_1: 5
spline_order_1: 3
learning_rate: 0.0015324287478969592
optimizer: adam
units_2: 26
grid_size_2: 4
spline_order_2: 4
units_3: 14
grid_size_3: 4
spline_order_3: 2
units_4: 16
grid_size_4: 3
spline_order_4: 4


Modello 4
num_layers: 4
units_0: 22
grid_size_0: 4
spline_order_0: 4
units_1: 23
grid_size_1